In [1]:
import tensorflow as tf
from tensorflow import keras
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
from preprocess import datasets
import gradio as gr

In [2]:
image_arrays, labels = datasets()

# Convert labels to numerical format using LabelEncoder
label_encoder = LabelEncoder()
encoded_labels = label_encoder.fit_transform(labels)

num_classes = len(label_encoder.classes_)

In [3]:
# Split the data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(np.array(image_arrays), encoded_labels, test_size=0.2, random_state=42)

In [4]:
# Define the CNN model
model = keras.Sequential([
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(64, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Conv2D(128, (3, 3), activation='relu'),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax')  # num_classes is the number of unique labels
])

In [5]:
# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
class_names = ['Alpinia Galanga (Rasna)', 'Amaranthus Viridis (Arive-Dantu)', 'Artocarpus Heterophyllus (Jackfruit)', 'Azadirachta Indica (Neem)', 'Basella Alba (Basale)', 'Brassica Juncea (Indian Mustard)', 'Carissa Carandas (Karanda)', 'Citrus Limon (Lemon)', 'Ficus Auriculata (Roxburgh fig)', 'Ficus Religiosa (Peepal Tree)', 'Hibiscus Rosa-sinensis', 'Jasminum (Jasmine)', 'Mangifera Indica (Mango)', 'Mentha (Mint)', 'Moringa Oleifera (Drumstick)', 'Muntingia Calabura (Jamaica Cherry-Gasagase)', 'Murraya Koenigii (Curry)', 'Nerium Oleander (Oleander)', 'Nyctanthes Arbor-tristis (Parijata)', 'Ocimum Tenuiflorum (Tulsi)', 'Piper Betle (Betel)', 'Plectranthus Amboinicus (Mexican Mint)', 'Pongamia Pinnata (Indian Beech)', 'Psidium Guajava (Guava)', 'Punica Granatum (Pomegranate)', 'Santalum Album (Sandalwood)', 'Syzygium Cumini (Jamun)', 'Syzygium Jambos (Rose Apple)', 'Tabernaemontana Divaricata (Crape Jasmine)', 'Trigonella Foenum-graecum (Fenugreek)']

print(class_names)

In [6]:
# Train the model
model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
46/46 [==============================] - 59s 1s/step - loss: 65.7625 - accuracy: 0.1465 - val_loss: 2.4009 - val_accuracy: 0.3460
Epoch 2/10
46/46 [==============================] - 55s 1s/step - loss: 1.6899 - accuracy: 0.5307 - val_loss: 1.5314 - val_accuracy: 0.5313
Epoch 3/10
46/46 [==============================] - 55s 1s/step - loss: 0.6759 - accuracy: 0.8099 - val_loss: 1.0385 - val_accuracy: 0.7193
Epoch 4/10
46/46 [==============================] - 58s 1s/step - loss: 0.3030 - accuracy: 0.9135 - val_loss: 1.2602 - val_accuracy: 0.6975
Epoch 5/10
46/46 [==============================] - 60s 1s/step - loss: 0.1670 - accuracy: 0.9462 - val_loss: 1.2083 - val_accuracy: 0.7221
Epoch 6/10
46/46 [==============================] - 59s 1s/step - loss: 0.1138 - accuracy: 0.9680 - val_loss: 1.4391 - val_accuracy: 0.7166
Epoch 7/10
46/46 [==============================] - 59s 1s/step - loss: 0.0441 - accuracy: 0.9898 - val_loss: 1.3395 - val_accuracy: 0.7629
Epoch 8/10
46/46 [=

In [7]:
model.evaluate(X_test, y_test)

12/12 [==============================] - 3s 270ms/step - loss: 1.4361 - accuracy: 0.7275


[1.4361485242843628, 0.7275204062461853]

In [ ]:
def predict_image(img):
  img_4d=img.reshape(-1,224,224,3)
  prediction=model.predict(img_4d)[0]
  return {class_names[i]: float(prediction[i]) for i in range(5)}

In [ ]:
image = gr.inputs.Image(shape=(224,224))
label = gr.outputs.Label(num_top_classes=6)

gr.Interface(fn=predict_image, inputs=image, outputs=label,interpretation='default').launch(debug='True')

In [8]:
model.save('E:\PlantIdentification\identification_model.keras')